In [ ]:
import math
import matplotlib.pyplot as plt
import keras
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from keras.callbacks import ReduceLROnPlateau 
from matplotlib import pyplot

from math import sqrt
from sklearn.metrics import mean_squared_error

from google.colab import drive
drive.mount('/content/drive')

#import LMPs
#df = pd.read_csv('/content/drive/MyDrive/CS229_Project/pacifica_oct_lmp.csv')
df = pd.read_csv('/content/drive/MyDrive/CS229_Project/bellehaven_2019.csv')

#import temperature data
temp = pd.read_csv('/content/drive/MyDrive/CS229_Project/merced_temp.csv')
images_dir = '/content/drive/MyDrive/CS229_Project'


In [ ]:

def train_test_split(df, temps, train_test_split=0.8):
  num_train = round(train_test_split * df.shape[0])
  training_set = np.zeros((num_train,2))
  training_set[:num_train,0] = df.iloc[:num_train, 1].values
  training_set[:num_train,1] = temps[:num_train]
  test_set = np.zeros((len(temps)-num_train,2))
  test_set[:,0] = df.iloc[num_train:, 1].values
  test_set[:,1] = temps[num_train:]
  return training_set, test_set, num_train

def resize_scale(training_set, test_set):
  #perform the feature scaling using the minmaxscaler from sklearn
  minmaxscale = MinMaxScaler(feature_range = (0, 1))
  scaled_train = minmaxscale.fit_transform(training_set)
  return scaled_train, minmaxscale


def data_shift(scaled_train, num_train, sequence_length=40, shift=1, num_features=2):
  #extract x train from price and temperature data and y train from the price data, shift by 1 
  X_train = np.zeros((num_train-sequence_length,sequence_length-shift,num_features))
  for i in range(sequence_length, num_train):
      X_train[i-sequence_length,:,0] = scaled_train[i-(sequence_length-shift):i, 0]
      X_train[i-sequence_length,:,1] = scaled_train[i-(sequence_length-shift):i, 1]
  y_train = scaled_train[sequence_length:num_train,0]
  return X_train, y_train

def LSTM_model(X_train):
  #define LSTM model with keras
  model = Sequential()

  # LSTM layer 1 and dropout regularization
  model.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], X_train.shape[2])))
  model.add(Dropout(0.2))

  # LSTM layer 2 and dropout regularization
  model.add(LSTM(units = 50, return_sequences = True))
  model.add(Dropout(0.2))

  # LSTM layer 3 and dropout regularization
  model.add(LSTM(units = 50))
  model.add(Dropout(0.2))

  # Output layer
  model.add(Dense(units = 1))

  #compile the model
  model.compile(optimizer = 'adam', loss = 'mean_squared_error')
  learning_rate = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=100)
  return model, learning_rate

def shift_test(df, temps, num_train,  minmaxscale, sequence_length=40,shift=1, num_features=2):
  #construct entire dataset
  dataset_total = np.zeros((len(temps),2))
  dataset_total[:,0] = df.iloc[:, 1].values
  dataset_total[:,1] = temps[:]

  #construct test set
  test_set = np.zeros((len(temps)-num_train,2))
  test_set[:,0] = df.iloc[num_train:, 1].values
  test_set[:,1] = temps[num_train:]

  test_in = dataset_total[len(dataset_total) - len(test_set) - sequence_length:]
  test_in = minmaxscale.transform(test_in)

  #shift test set by lags
  X_test = np.zeros((len(test_set),sequence_length-shift,num_features))
  for i in range(sequence_length, len(test_set)+sequence_length):
      X_test[i-sequence_length,:,0] = test_in[i-(sequence_length-shift):i, 0]
      X_test[i-sequence_length,:,1] = test_in[i-(sequence_length-shift):i, 1]

  #make data arrays
  X_test = np.array(X_test)
  return X_test, test_set

def inverse_predictions(X_test):
  predicted_price = model.predict(X_test)
  predicted_price_t = np.zeros((len(predicted_price),2))
  new_temp = np.resize(predicted_price,(len(predicted_price),))
  predicted_price_t[:,0] = new_temp
  predicted_price_t[:,1] = new_temp
  predicted_price = minmaxscale.inverse_transform(predicted_price_t)
  return predicted_price

def RMSE(actual, predicted):
  # calculate RMSE
  rmse = sqrt(mean_squared_error(actual, predicted))
  print('Test RMSE: %.3f' % rmse)
  return rmse

def plot_func(actual, predicted):
  # Plotting predicted vs actual price
  plt.plot(actual[:,0], label = 'Real Price')
  plt.plot(predicted[:,0], color='red', label = 'Predicted Price')
  plt.title('LSTM LMP Price Prediction')
  plt.xlabel('Time [days]')
  plt.xticks(ticks=np.arange(0, len(predicted), 96), labels=[4*int(x) for x in np.arange(0, len(predicted), 24)/24])
  plt.ylabel('Price [$]')
  plt.legend()
  #plt.savefig(f"{images_dir}/LSTM_LMP_twofeatures_100epochs_lag1.png", dpi=500) 
  plt.show()

def plot_loss(model_params):
  # Plot the losses over the epochs
  plt.clf()
  plt.plot(model_params.history['loss'])
  plt.plot(model_params.history['val_loss'])
  plt.title('Model Loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Training Data', 'Validation Data'], loc='upper right')

  images_dir = '/content/drive/MyDrive/CS229_Project'
  #plt.savefig(f"{images_dir}/LSTM_loss_2features_100epochs_lag1.png", dpi=500) 

  plt.show()

def plot_temp(temp_test):
  #plot temperature data for visualization purposes
  plt.clf()
  plt.plot(temp_test[:,1])

  plt.ylabel('temp (C)')
  plt.xlabel('time (hrs)')
  plt.legend(['Temp'], loc='upper right')

  plt.show()


#preprocess temperatures from dataset
temps = temp.iloc[:,2].values
temps[temps<-20] = 0

training_set, test_set, num_train = train_test_split(df, temps)
scaled_train, minmaxscale = resize_scale(training_set, test_set)
X_train, y_train = data_shift(scaled_train, num_train)
model, learning_rate = LSTM_model(X_train)
epochs=100
batch_size=32
validation=.05
model_params=model.fit(X_train, y_train, batch_size, epochs, validation_split = validation, callbacks = [learning_rate])
X_test, y_test=shift_test(df, temps, num_train, minmaxscale)
predicted=inverse_predictions(X_test)
RMSE(y_test, predicted)
plot_func(y_test, predicted)
plot_loss(model_params)
plot_temp(y_test)
